<a href="https://colab.research.google.com/github/josephmailil1/Speciale---Stibo/blob/main/Agent%20Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [35]:
from google.colab import userdata
openai_api_key = userdata.get('OpenAI_Key') #use the name of your colab key

In [3]:
import pandas as pd

shap = pd.read_csv("shap_noincome.csv")
data = pd.read_csv("basedata.csv")

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

tool = PythonAstREPLTool(locals={"data": data, "shap": shap})
tool.invoke("data['age'].mean()")

38.54927397361244

In [6]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
llm_with_tools.invoke(
    "I have a dataframe 'data' and want to know the correlation between the 'age' and 'capital-gain' columns"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_L9IQm6jjQmf93VbM3C3NdTcG', 'function': {'arguments': '{"query":"data[[\'age\', \'capital-gain\']].corr()"}', 'name': 'python_repl_ast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 126, 'total_tokens': 141}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-c12a77b4-dff1-410c-a299-fb5acbc26e9d-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "data[['age', 'capital-gain']].corr()"}, 'id': 'call_L9IQm6jjQmf93VbM3C3NdTcG'}])

In [7]:
from langchain.output_parsers import JsonOutputToolsParser

In [8]:
parser = JsonOutputToolsParser(tool = tool)

In [9]:
system = f"""You have access to a pandas dataframe 'data'. This dataframe contains  \
Here is the output of `data.head().to_markdown()`:

```
{data.head().to_markdown()}
```
You are also given the pandas dateframe 'shap'. Containing shap values for the test set obsevations. \
The shap values shows how each feature impacts the y-variable income on a local prediction level.
```
{shap.head().to_markdown()}
```
Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

In [10]:
chain = prompt | llm_with_tools | parser  # noqa
chain.invoke({"question": "What is the most important feature value for the first obsevation?'"})

[{'args': {'query': 'shap.iloc[0].idxmax()'}, 'type': 'python_repl_ast'}]

In [11]:
data['age'].corr(data['capital-gain'])

0.08506786648520717

In [12]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe 'data'. This is the dataframe containing actual observations and values of the dataset \
Here is the output of 'data.head().to_markdown()':

```
{data.head().to_markdown()}
```
You are also given the pandas dateframe 'shap'. Containing shap values for the test set obsevations. \
The shap values shows how each feature impacts the y-variable income on a local prediction level. The actual Y-variable is not available in this dataset. \
When asked to explain something, use the shap values for the observations to substantiate your answer.
```

{shap.head().to_markdown()}
```

The dataset contains 13K observations of descriptive data on individuals, with the prediction task being to determine whether a person makes over $50K a year.

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Follow these steps: \
  1. Generete the python code you want to use. \
  2. Execute The Python Code \
  3. Answer the Question based on the exececuted Python Code in Step 2 \
  4. Return both the answer and the Python Code you execeuted marked in []."""


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser )
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["response"])
)

In [13]:
from langchain.memory import ConversationBufferMemory
# This is do give the bot memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [14]:
def call_chain(user_question):
    answer = chain.invoke({"question": user_question})
    return answer['response']

# Interface - Gradio

In [15]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.9 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=e2960a97999ded7edb493951ef7a0a86fb1719bb5dbf577e73c28185add9e065
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: typer
   

In [16]:
import gradio as gr

In [17]:
demo = gr.Interface(
    fn=call_chain,
    inputs=["text"],
    outputs=["text"]
)

In [18]:
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Missing file: /usr/local/lib/python3.10/dist-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.10/dist-packages/gradio


<IPython.core.display.Javascript object>

In [19]:
shap.loc[0, ['marital-status_Married-civ-spouse', 'education-num', 'occupation_Other-service', 'sex_Male']]

marital-status_Married-civ-spouse    0.081040
education-num                        0.102855
occupation_Other-service             0.003209
sex_Male                             0.011475
Name: 0, dtype: float64

In [20]:
shap.loc[0].nlargest(5)

education-num                        0.102855
marital-status_Married-civ-spouse    0.081040
education_Bachelors                  0.052379
relationship_Not-in-family           0.028823
marital-status_Never-married         0.027377
Name: 0, dtype: float64

In [21]:
important_features = shap.iloc[0].sort_values(ascending=False)
print(important_features)

education-num                        0.102855
marital-status_Married-civ-spouse    0.081040
education_Bachelors                  0.052379
relationship_Not-in-family           0.028823
marital-status_Never-married         0.027377
                                       ...   
occupation_Exec-managerial          -0.013202
native-country_United-States        -0.014199
occupation_Transport-moving         -0.015753
capital-gain                        -0.018275
hours-per-week                      -0.033657
Name: 0, Length: 96, dtype: float64


In [22]:
shap['marital-status_Married-civ-spouse'].mean()

-0.004628922169480658

In [31]:
from langchain.document_loaders import CSVLoader # used to load propriatory data
from langchain.vectorstores import DocArrayInMemorySearch # load an in-memory vector store
from IPython.display import display, Markdown # common utilities for displaying information in notebooks.

In [24]:
loader = CSVLoader("basedata.csv")
basedata = loader.load()

In [29]:
loader = CSVLoader("shap_noincome.csv")
shap = loader.load()

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [36]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)

In [42]:
!pip install --quiet docarray
!pip install --quiet tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 5.3 MB/s eta 0:00:00


In [44]:
db = DocArrayInMemorySearch.from_documents(
    basedata,
    embeddings
)

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


KeyboardInterrupt: 

In [43]:
retriever = db.as_retriever()

NameError: name 'db' is not defined

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
from operator import itemgetter

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

system = f"""You have access to a pandas dataframe 'data'. This is the dataframe containing actual observations and values of the dataset \
Here is the output of 'data.head().to_markdown()':

```
{data.head().to_markdown()}
```
You are also given the pandas dateframe 'shap'. Containing shap values for the test set obsevations. \
The shap values shows how each feature impacts the y-variable income on a local prediction level. The actual Y-variable is not available in this dataset. \
When asked to explain something, use the shap values for the observations to substantiate your answer.
```

{shap.head().to_markdown()}
```

The dataset contains 13K observations of descriptive data on individuals, with the prediction task being to determine whether a person makes over $50K a year.

Given a user question, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Follow these steps: \
  1. Generete the python code you want to use. \
  2. Execute The Python Code \
  3. Answer the Question based on the exececuted Python Code in Step 2 \
  4. Return both the answer and the Python Code you execeuted marked in []."""


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system,
        ),
        ("human", "{question}"),
        # This MessagesPlaceholder allows us to optionally append an arbitrary number of messages
        # at the end of the prompt using the 'chat_history' arg.
        MessagesPlaceholder("chat_history", optional=True),
    ]
)


def _get_chat_history(x: dict) -> list:
    """Parse the chain output up to this point into a list of chat history messages to insert in the prompt."""
    ai_msg = x["ai_msg"]
    tool_call_id = x["ai_msg"].additional_kwargs["tool_calls"][0]["id"]
    tool_msg = ToolMessage(tool_call_id=tool_call_id, content=str(x["tool_output"]))
    return [ai_msg, tool_msg]


chain = (
    RunnablePassthrough.assign(ai_msg=prompt | llm_with_tools)
    .assign(tool_output=itemgetter("ai_msg") | parser )
    .assign(chat_history=_get_chat_history)
    .assign(response=prompt | llm | StrOutputParser())
    .pick(["response"])
)